## Common code and data loading

In [1]:
#ONLY RUN THIS IF THE GPU SERVER IS BUSY
#USES THE SLOWASS CPU INSTEAD

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [1]:
#Run this to choose 2nd GPU

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
# Run this if the following error is thrown when training/testing models:

# Error : Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, 
# so try looking to see if a warning log message was printed above.

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config_tf()


def config_tf():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    set_session(sess)
    
config_tf()

Using TensorFlow backend.


In [2]:

import SimpleITK as sitk
import os, sys
sys.path.insert(1, './Models/Resnet-3D')
from resnet3d3 import Resnet3DBuilder
import resnet3d3

import numpy as np
import random
import keras
from sklearn.model_selection import train_test_split
import math
from keras.callbacks import EarlyStopping

#these are imported or the transferring models are not deserialised properly
import conv4d
from keras.regularizers import l2

In [3]:
%load_ext autoreload
%autoreload 2

#if this is not here, the loaded notebook will not detect ensuing changes in imported numpy scripts 

In [4]:
#load training data from disk

pet_data, ct_data, centre_data, size_data, mask_data, ct_radiomics_data, pet_radiomics_data = {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, \
                                                        {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, \
                                                        {'train':{}, 'test':{}}, {'train':{}, 'test':{}}

for dataset in ['HeadNeckCancer', 'BreastCancer']:
    pet_files = []
    ct_files = []
    centre_files = []
    size_files = []
    mask_files = []
    pet_radiomics_files = []
    ct_radiomics_files = []
    
    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/PET'):
        for name in files:
            file_path = os.path.join(root, name)
            pet_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/CT'):
        for name in files:
            file_path = os.path.join(root, name)
            ct_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/MaskCentres'):
        for name in files:
            file_path = os.path.join(root, name)
            centre_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/MaskSizes'):
        for name in files:
            file_path = os.path.join(root, name)
            size_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/Mask'):
        for name in files:
            file_path = os.path.join(root, name)
            mask_files.append(file_path)
            
    for root, dirs, files in os.walk('/home/jzhe0882/Radiomics/' + dataset + '/CT'):
        for name in files:
            file_path = os.path.join(root, name)
            ct_radiomics_files.append(file_path)
            
    for root, dirs, files in os.walk('/home/jzhe0882/Radiomics/' + dataset + '/PET'):
        for name in files:
            file_path = os.path.join(root, name)
            pet_radiomics_files.append(file_path)

    pet_files = sorted(pet_files)
    ct_files = sorted(ct_files)
    centre_files = sorted(centre_files)
    centres = [np.load(c) for c in centre_files] #can load all of these into memory (other volumes are too large)
    size_files = sorted(size_files)
    bounding_box_sizes = [np.load(s) for s in size_files] #can also load all of these into memory
    mask_files = sorted(mask_files)
    pet_radiomics_files= sorted(pet_radiomics_files)
    ct_radiomics_files = sorted(ct_radiomics_files)

    #Inputs are PET/CT data, outputs are centres, sizes, masks
    X_train, X_test, y_train, y_test = train_test_split(list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files)), 
                                                        list(zip(centres, bounding_box_sizes, mask_files)), 
                                                        test_size=0.33, shuffle=True, random_state=9)
    
    pet_data['train'][dataset], ct_data['train'][dataset], pet_radiomics_data['train'][dataset], ct_radiomics_data['train'][dataset] = zip(*X_train)
    pet_data['test'][dataset], ct_data['test'][dataset], pet_radiomics_data['test'][dataset], ct_radiomics_data['test'][dataset] = zip(*X_test)
    centre_data['train'][dataset], size_data['train'][dataset], mask_data['train'][dataset] = zip(*y_train)
    centre_data['test'][dataset], size_data['test'][dataset], mask_data['test'][dataset] = zip(*y_test)

    print(pet_data['train'][dataset][0], ct_data['train'][dataset][0], pet_radiomics_data['train'][dataset][0], ct_radiomics_data['train'][dataset][0],
          centre_data['train'][dataset][0], size_data['train'][dataset][0], mask_data['train'][dataset][0])
    
    print(pet_data['test'][dataset][0], ct_data['test'][dataset][0], pet_radiomics_data['test'][dataset][0], ct_radiomics_data['test'][dataset][0],
          centre_data['test'][dataset][0], size_data['test'][dataset][0], mask_data['test'][dataset][0])

/home/jzhe0882/numpydata/HeadNeckCancer/PET/HN-CHUS-048.npy /home/jzhe0882/numpydata/HeadNeckCancer/CT/HN-CHUS-048.npy /home/jzhe0882/Radiomics/HeadNeckCancer/PET/HN-CHUS-048.npy /home/jzhe0882/Radiomics/HeadNeckCancer/CT/HN-CHUS-048.npy [ 64  49 105] [9 8 7] /home/jzhe0882/numpydata/HeadNeckCancer/Mask/HN-CHUS-048.npy
/home/jzhe0882/numpydata/HeadNeckCancer/PET/HN-HGJ-035.npy /home/jzhe0882/numpydata/HeadNeckCancer/CT/HN-HGJ-035.npy /home/jzhe0882/Radiomics/HeadNeckCancer/PET/HN-HGJ-035.npy /home/jzhe0882/Radiomics/HeadNeckCancer/CT/HN-HGJ-035.npy [70 38 61] [16 14 33] /home/jzhe0882/numpydata/HeadNeckCancer/Mask/HN-HGJ-035.npy
/home/jzhe0882/numpydata/BreastCancer/PET/10368550.npy /home/jzhe0882/numpydata/BreastCancer/CT/10368550.npy /home/jzhe0882/Radiomics/BreastCancer/PET/10368550.npy /home/jzhe0882/Radiomics/BreastCancer/CT/10368550.npy [46 58 62] [3 3 4] /home/jzhe0882/numpydata/BreastCancer/Mask/10368550.npy
/home/jzhe0882/numpydata/BreastCancer/PET/10359092.npy /home/jzhe0882/

In [5]:


# (x,y,z, a) + (x,y,z, b) => (x,y,z, a+b)
def zip_np_volumes(vol_a, vol_b):
    vol_a = np.expand_dims(np.array(vol_a), axis=-1)
    vol_b = np.expand_dims(np.array(vol_b), axis=-1)
    return np.concatenate((vol_a, vol_b), axis=-1)

def combine_radiomics_data(source_vol, radiomics_vol):
    source_vol = np.expand_dims(source_vol, axis=-1)
    return np.concatenate((source_vol, radiomics_vol), axis=-1)

def normalise_volume(vol):
    std = np.std(vol)
    if np.isclose(std, 0):
        std = 1
    vol = np.divide(vol - np.mean(vol), std)
    return vol

volume_size = np.array([128,128,128], dtype=int)
def shift_centre(centre, shift_weight, seed=None):
    np.random.seed(seed) 
    
    centre = np.random.randn(3) * shift_weight * np.array([19.69, 4.24, 13.83]) + centre #std taken from NumpyAnalysis
    centre = np.maximum(centre, [0,0,0]) #keep centre within image bounds
    centre = np.minimum(centre, volume_size)
    return centre.astype(int)

In [6]:
#source_volume is a 3-dim (w,h,d) or 4-dim array (w,h,d,channels)
#generates a bounding box volume given the parameters
#centre is assumed to be relative to the source volume and rounded down
#extents is size of the bounding_box / 2
def get_bounding_box(source_volume, centre, extents):
    extents_ceil = np.ceil(extents).astype(int)
    extents_floor = extents.astype(int)
    centre = np.rint(centre).astype(int)
    
    maxima = centre + extents_ceil
    minima = centre - extents_floor
    
    #keep bounding box dimensions within the mask dimensions
    maxima = np.minimum(maxima, np.array(source_volume.shape)[:3]).astype(int)
    minima = np.maximum(minima, [0,0,0]).astype(int) 
        
    bounding_box_values = source_volume[minima[0]:maxima[0],
                                      minima[1]:maxima[1],
                                      minima[2]:maxima[2]]
    
    relative_centre = extents_floor
    relative_maxima = relative_centre + maxima - centre
    relative_minima = relative_centre + minima - centre
    
    bbox_shape = tuple(extents_ceil + extents_floor)
    if len(source_volume.shape) == 4:
        bbox_shape += (source_volume.shape[3],)
    bounding_box = np.zeros(bbox_shape)
    
    bounding_box[relative_minima[0]:relative_maxima[0],
                relative_minima[1]:relative_maxima[1],
                relative_minima[2]:relative_maxima[2]] = bounding_box_values
    
    return bounding_box

#translates the bounding box so that its values have a new reference centre
def align_bounding_box(bounding_box, box_centre, target_centre):
    displacement = (target_centre - box_centre).astype(int)
    new_box = np.copy(bounding_box)
    
    #boxes to be shifted forward have a trails of zeroes at the end of the array
    #boxes to be shifted backward have a trails of zeroes at the beginning of the array
    if displacement[0] < 0:
        new_box[displacement[0]:, :, :] = 0
    else:
        new_box[:displacement[0], :, :] = 0
                      
    if displacement[1] < 0:
        new_box[:, displacement[1]:, :] = 0
    else:
        new_box[:, :displacement[1], :] = 0
    
    if displacement[2] < 0:
        new_box[:, :, displacement[2]:] = 0
    else:
        new_box[:, :, :displacement[2]] = 0
    
    new_box = np.roll(new_box, -displacement, axis=(0,1,2))
    return new_box

#resample bounding_box to target_size while keeping aspect ratios
def resize_bounding_box(bounding_box, target_size, mode):
    size_ratio = np.amin(np.divide(target_size, bounding_box.shape))
    size_ratio_index = np.argmin(np.divide(target_size, bounding_box.shape))
    
    #the resampling here is slightly inaccurate since new_size is rounded to the nearest int
    bounding_box = sitk.GetImageFromArray(np.transpose(bounding_box))
    new_size = np.rint(size_ratio * np.array(bounding_box.GetSize()))
    new_spacing = tuple(np.multiply(np.divide(bounding_box.GetSpacing(), size_ratio), np.divide(new_size, new_size+1)))
    #additionally multiply spacing by size/(size+1) to fill out an extra row of values
        
    assert new_size[size_ratio_index] == target_size[size_ratio_index]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(bounding_box)
    resampler.SetSize((int(new_size[0]), int(new_size[1]), int(new_size[2])))
    resampler.SetOutputSpacing(new_spacing)

    if mode == 'mask':
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
        
    new_box = np.transpose(sitk.GetArrayFromImage(resampler.Execute(bounding_box)))
    
    #pads the surroundings with 0's
    return get_bounding_box(new_box, 0.5 * new_size, 0.5 * target_size)
            


## Detection Model

In [11]:
#generator for (ct,pet)->centre prediction models
def detection_generator(mode='train', dataset='HeadNeckCancer', shuffle=True, normalise=True, use_radiomics=True, batch_size=4):
    
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, shuffle, normalise, use_radiomics, batch_size):
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres = zip(*z)

            ct_pet_batch = []
            centre_batch = []
            
            for i in range(len(pet_files)):
                ct, pet = np.load(ct_files[i]), np.load(pet_files[i])
                
                if use_radiomics:
                    ct_radiomics = np.load(ct_radiomics_files[i])
                    ct = combine_radiomics_data(ct, ct_radiomics)
                    
                    pet_radiomics = np.load(pet_radiomics_files[i])
                    pet = combine_radiomics_data(pet, pet_radiomics)
            
                else:
                    ct = np.expand_dims(ct, axis=-1)
                    pet = np.expand_dims(pet, axis=-1)
                    
                ct_pet = np.concatenate((ct, pet), axis=-1)
                    
                if normalise:                    
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                centre_batch.append(centres[i])

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(centre_batch)

                    ct_pet_batch.clear()
                    centre_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(centre_batch)
    
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  centre_data[mode][dataset], shuffle, normalise, use_radiomics, batch_size)

# generator for (ct,pet,centre)->(centre,size) prediction models
def localisation_generator(mode='train', dataset='HeadNeckCancer', shuffle=True, normalise=True, use_radiomics=True, batch_size=4, 
                           shift_weight=0.7, region_radius=[16,16,16]):

    
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, shuffle, normalise, use_radiomics, batch_size):
        val_seeds = list(range(len(pet_files)))

        while True:
            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, val_seeds))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, val_seeds = zip(*z)

            ct_pet_batch = []
            shift_centre_batch = []
            true_centre_size_batch = []
            
            for i in range(len(pet_files)):
                ct, pet = np.load(ct_files[i]), np.load(pet_files[i])
                
                if use_radiomics:
                    ct_radiomics = np.load(ct_radiomics_files[i])
                    ct = combine_radiomics_data(ct, ct_radiomics)
                    
                    pet_radiomics = np.load(pet_radiomics_files[i])
                    pet = combine_radiomics_data(pet, pet_radiomics)
            
                else:
                    ct = np.expand_dims(ct, axis=-1)
                    pet = np.expand_dims(pet, axis=-1)
                    
                ct_pet = np.concatenate((ct, pet), axis=-1)
                    
                if normalise:                    
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                if mode == 'test': #make the validation generator generate predictable centres
                    seed = val_seeds[i]
                elif mode == 'train':
                    seed = None
                
                shifted_centre = shift_centre(centres[i], shift_weight, seed)
                ct_pet = get_bounding_box(ct_pet, shifted_centre, np.array(region_radius))
                                
                ct_pet_batch.append(ct_pet)
                shift_centre_batch.append(shifted_centre)
                true_centre_size_batch.append([centres[i], sizes[i]])

                if len(ct_pet_batch) == batch_size:
                    yield [np.array(ct_pet_batch), np.array(shift_centre_batch)], np.array(true_centre_size_batch)

                    ct_pet_batch.clear()
                    shift_centre_batch.clear()
                    true_centre_size_batch.clear()

            if len(ct_pet_batch) > 0:
                yield [np.array(ct_pet_batch), np.array(shift_centre_batch)], np.array(true_centre_size_batch)
    
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  centre_data[mode][dataset], size_data[mode][dataset], shuffle, normalise, use_radiomics, batch_size)

In [66]:
#test to see if generator works

batch_size = 10
dataset = 'HeadNeckCancer'
mode = 'test'
test_generator = localisation_generator(mode=mode, dataset=dataset, batch_size=batch_size, shuffle=True, use_radiomics=False)
print('{} batches of {} samples taken over {} total samples'.format(
    math.ceil(len(ct_data[mode][dataset])/batch_size), batch_size, len(ct_data[mode][dataset])))

for i in range(1):#math.ceil(len(ct_data[mode][dataset])/batch_size)):
    ctpet, centresize = next(test_generator)
    print(i, ctpet[0].shape, ctpet[1].shape, centresize.shape)


10 batches of 10 samples taken over 98 total samples
0 (10, 32, 32, 32, 2) (10, 3) (10, 2, 3)


In [18]:
import tensorflow as tf
import keras.backend as K
import keras.losses

#y_true, y_pred in the format (batch_size, centre/size, image axes)
#GENERALISED IoU :o o_0 ermagerd
def GIoU_helper(IoU_only=False):
    
    def GIoU(y_true, y_pred):
        num_axes=3

        #calculate volume of cuboid given its max and min (x,y,z) coordinates
        def calculate_volume(max_coords, min_coords):
            #K.ones throws an uninitialised variable exception something something
            vols = K.maximum(tf.constant([0.0]), max_coords[:, 0] - min_coords[:, 0]) #max_coords.shape[0] = batch size

             # if min_coord >= max_coord for some sample in the batch, then that sample has non-overlapping boxes
                # assign 0 volume to such samples
            for i in range(1, num_axes):
                vols = vols * K.maximum(tf.constant([0.0]), max_coords[:, i] - min_coords[:, i])

            return vols

        #extract values from tensors
        centre_pred = K.cast(y_pred[:, 0, :], dtype='float32')
        size_pred = K.cast(y_pred[:, 1, :], dtype='float32')
        centre_true = K.cast(y_true[:, 0, :], dtype='float32')
        size_true = K.cast(y_true[:, 1, :], dtype='float32')

        #obtain the min/max coordinates of the bounding boxes
        pred_max = centre_pred + 0.5 * size_pred
        pred_min = centre_pred - 0.5 * size_pred
        true_max = centre_true + 0.5 * size_true
        true_min = centre_true - 0.5 * size_true

        intersect_min = K.maximum(pred_min, true_min) # "small" corner of overlapping box
        intersect_max = K.minimum(pred_max, true_max) # "large" corner of overlapping box
        intersect_volume = calculate_volume(intersect_max, intersect_min)

        union_volume = calculate_volume(pred_max, pred_min) + calculate_volume(true_max, true_min) - intersect_volume

        hull_min = K.minimum(pred_min, true_min)
        hull_max = K.maximum(pred_max, true_max)
        hull_volume = calculate_volume(hull_max, hull_min)

        IoU = intersect_volume / union_volume
        extra = (hull_volume - union_volume) / hull_volume

        if IoU_only:
            return -(IoU)
        else:
            return -(IoU - extra)
    
    return GIoU

def IoU(y_true, y_pred):
    return GIoU_helper(IoU_only=True)(y_true, y_pred)

def hybrid_GIoU_MSE(y_true, y_pred, delta=50):
    giou = GIoU_helper()(y_true, y_pred)
    mse = K.mean(keras.losses.mean_squared_error(y_true, y_pred))
    return delta * giou + mse
    

with tf.Session() as sess:

    t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
    
    u = tf.constant([[[1, 1, 1], [2, 2, 2]], 
                     [[1, 1, 1], [2, 2, 2]]])
    v = tf.constant([[[1, 1, 1], [2, 2, 6]],
                    [[1, 2, 1], [2, 2, 4]]])
    
    a = tf.slice(t, [1, 0, 0], [1, 1, 3])  # [[[3, 3, 3]]]
    b = tf.slice(t, [1, 0, 0], [1, 2, 3])  # [[[3, 3, 3],
                                       #       [4, 4, 4]]]
    c = tf.slice(t, [1, 0, 0], [2, 1, 3])  # [[[3, 3, 3]],
                                           #  [[5, 5, 5]]]
        
    d = K.expand_dims(t, axis=1)
        
    booleans = tf.constant([False, False])
          
    centres = tf.slice(t, [0,0,0], [3,1,3])
    sizes = tf.slice(t, [0,1,0], [3,1,3])
    
    print(sess.run(t).shape, sess.run(d).shape)
    
    print(sess.run(GIoU_helper()(u, v)))
    
    print(sess.run(IoU(u, v)))
    
    print(sess.run(keras.losses.mean_squared_error(u, v)))
    

(3, 2, 3) (3, 1, 2, 3)
[-0.33333334 -0.03333333]
[-0.33333334 -0.2       ]
[[0 5]
 [0 1]]


In [13]:
# Evaluate detection model

from sklearn.metrics import mean_squared_error


#print(detection_model.evaluate_generator(validation_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size)))

def evaluate_detection_model(model, dataset, function='detection', batch_size=4):
    mode = 'test'
    
    if function == 'detection':
        test_generator = detection_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size, use_radiomics=False)    
        detected_centres = model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))
        
        return str(mean_squared_error(detected_centres, centre_data[mode][dataset], multioutput='raw_values'))
    
    elif function == 'localisation':
        test_generator = localisation_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size, use_radiomics=False)    
        detected_centresizes = model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))
        detected_centre_offsets = detected_centresizes[:, 0, :]
        detected_sizes = detected_centresizes[:, 1, :]

        #print(detected_centres.shape)
        centre_mse =  mean_squared_error(detected_centre_offsets, centre_data[mode][dataset], multioutput='raw_values')
        size_mse = mean_squared_error(detected_sizes, size_data[mode][dataset], multioutput='raw_values')
    
        return str(centre_mse) + str(size_mse)

    
#print(evaluate_detection_model(detection_model, 'BreastCancer', function='localisation'))


In [16]:
reset_keras()

956


In [ ]:
from keras.models import load_model
from keras import losses
import utils

#cycles_per_epoch = how many times the entire training set should be cycled over for each epoch
#total_cycles = how many times the entire training set should be cycled in total

def train_detection_model(dataset, save_model=True, batch_size=4, total_epochs=100, normalise=True, shuffle=True,
                          use_radiomics=True, transferring_model=None, loss=losses.mean_squared_error, es_patience=20,
                         function='detection'):

    def get_model_name():
        return '{} {} batch={} cycles={} shuffle={} normalise={} use_radiomics={} transferred={} {}.h5'.format(
            function, dataset, batch_size, total_epochs, shuffle, normalise, use_radiomics, transferring_model!=None, loss.__name__)
        
    #define input shapes
    if use_radiomics:
        num_channels = 16
    else:
        num_channels = 2
        
    if function == 'detection':
        detection_input_shape = (128, 128, 128, num_channels)
        generator_funct = detection_generator
            
    elif function == 'localisation':
        detection_input_shape = [(32, 32, 32, num_channels), (3,)]
        generator_funct = localisation_generator

    train_generator = generator_funct(mode='train', dataset=dataset, batch_size=batch_size, shuffle=shuffle, normalise=normalise,
                                          use_radiomics=use_radiomics)
    validation_generator = generator_funct(mode='test', dataset=dataset, batch_size=batch_size, shuffle=False, normalise=normalise, 
                                               use_radiomics=use_radiomics)

    #determine whether to use old model or build new one from scratch
    if transferring_model is not None:
        detection_model = transferring_model
        #for source_layer, target_layer in zip(transferring_model.layers, detection_model.layers):
         #   target_layer.set_weights(source_layer.get_weights())
    else:
        if function == 'localisation':
            resnet_function = Resnet3DBuilder.build_resnet_34
            metrics = [IoU, GIoU_helper(), 'mean_squared_error']
            
        elif function == 'detection':
            resnet_function = Resnet3DBuilder.build_resnet_18
            metrics = []

        
        detection_model = resnet_function(detection_input_shape, [3,3], mode=function, reg_factor=1e-8)
        detection_model.compile(optimizer='adam',
                          loss=loss, metrics=metrics)
        
    get_best = utils.GetBest(monitor='val_loss', verbose=0, mode='min')
    history = detection_model.fit_generator(train_generator, validation_data=validation_generator, verbose=1,
                                  validation_steps=math.ceil(len(ct_data['test'][dataset])/batch_size),
                                  steps_per_epoch=math.ceil(len(ct_data['train'][dataset])/batch_size), 
                                  epochs=total_epochs,
                                    callbacks=[get_best])

    loss_hist = history.history['loss']
    val_loss_hist = history.history['val_loss']
    best_loss = np.amin(val_loss_hist)
    best_mse = evaluate_detection_model(detection_model, dataset, function=function) #np.amin(history.history['val_mean_squared_error'])
    
    if function == 'localisation':
        best_giou = np.amin(history.history['val_GIoU'])
        best_iou = np.amin(history.history['val_IoU'])
    else:
        best_giou = 'N/A'
        best_iou = 'N/A'
    
    with np.printoptions(precision=2):
        print('loss:', np.array(loss_hist))
        print('val_loss:', np.array(val_loss_hist))
        print('best_loss:', best_loss)
        print('best_mse:', best_mse)
        print('best_iou:', best_iou)
        print('best_giou:', best_giou)
        
    with open('Models/keras models/training_log.txt', 'a+') as log:
        log.write('{} \nbest loss/mse/giou/iou: {} {} {} {}\ntraining: {}\nvalidation: {}\n\n'.format(
            get_model_name(), best_loss, best_mse, best_giou, best_iou, loss_hist, val_loss_hist))
    
    if save_model:
        model_name = get_model_name()
        detection_model.save('Models/keras models/' + model_name)
        
    return detection_model
    
'''detection_model = train_detection_model(dataset='HeadNeckCancer', use_radiomics=False, total_epochs=200, save_model=True, function='detection',
                                   loss=losses.mean_squared_error)'''

'''transferring_model=load_model('Models/keras models/detection HeadNeckCancer batch=4 cycles=200 shuffle=True normalise=True use_radiomics=False transferred=False mean_squared_error.h5',
                             custom_objects={'jaccard_distance_loss' : jaccard_distance_loss,
                                    'dice_coef_loss': dice_coef_loss,
                                          'dice_metric': dice_metric,
                                          'dice_coef': dice_coef,
                                       'conv4d' : conv4d.conv4d,
                                    'l2': l2,
                                    'Resnet3DBuilder': Resnet3DBuilder,
                                    'tf' : tf,
                                    'upsample3d_helper' : resnet3d3.upsample3d_helper})
detection_model = train_detection_model(dataset='BreastCancer', use_radiomics=False, total_epochs=200, save_model=True, function='detection',
                                   loss=losses.mean_squared_error, transferring_model=transferring_model)'''



localisation_model = train_detection_model(dataset='BreastCancer', use_radiomics=False, total_epochs=200, save_model=True, function='localisation',
                                       loss=hybrid_GIoU_MSE, es_patience=190)
reset_keras()


In [ ]:
print(detected_centres[92], centre_data['test']['HeadNeckCancer'][92])


## Segmentation Model

In [22]:
#values derived from NumpyAnalysis.ipynb
#get_best_bounding_box()
maximal_bounding_volume = np.array([32,32,32])

#generator for (ct bounding box, pet bounding_box)-> mask bounding box prediction models
def mask_bounding_box_predictor_generator(mode='train', dataset='HeadNeckCancer', shift_centres=True, shift_sizes=True, shift_weight=0.7, 
                                          shuffle=False, normalise=True, use_radiomics=True, batch_size=4, region_radius=[16,16,16]):

            
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, mask_files, centres, sizes, 
               shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size):
        val_seeds = list(range(len(pet_files)))
        
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files, val_seeds))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files, val_seeds = zip(*z)
                                
            ct_pet_batch = []
            mask_batch = []
            
            for i in range(len(pet_files)):
                
                if mode == 'test': #make the validation generator generate predictable centres
                    seed = val_seeds[i]
                elif mode == 'train':
                    seed = None
                
                #randomly sample centres/sizes around the true centre/sizes to generate bounding boxes
                if shift_centres:
                    centre = shift_centre(centres[i], shift_weight, seed)
                else:
                    centre = centres[i]

                ct = get_bounding_box(np.load(ct_files[i]), centre, np.array(region_radius))
                pet = get_bounding_box(np.load(pet_files[i]), centre, np.array(region_radius))
                mask = get_bounding_box(np.load(mask_files[i]), centre, np.array(region_radius))

                ct_pet = zip_np_volumes(ct, pet)
                
                if use_radiomics:
                    ct_radiomics = np.load(ct_radiomics_files[i])
                    pet_radiomics = np.load(pet_radiomics_files[i])
                    
                    for axis in range(ct_radiomics.shape[-1]):
                        ct_radiomic_feature = get_bounding_box(ct_radiomics[:,:,:,axis], centre, np.array(region_radius))
                        pet_radiomic_feature = get_bounding_box(pet_radiomics[:,:,:,axis], centre, np.array(region_radius))
                        ct_radiomic_feature = np.expand_dims(ct_radiomic_feature, axis=-1)
                        pet_radiomic_feature = np.expand_dims(pet_radiomic_feature, axis=-1)
                        
                        ct_pet = np.concatenate((ct_pet, ct_radiomic_feature, pet_radiomic_feature), axis=-1)
                
                if normalise:
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                mask_batch.append(mask)
                
                #print(centre - centres[i])

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(mask_batch)
                    ct_pet_batch.clear()
                    mask_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(mask_batch)
                
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  mask_data[mode][dataset], centre_data[mode][dataset], size_data[mode][dataset],
                 shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size)

In [81]:
#check to see if generator works
batch_size = 10
mode = 'test'
dataset = 'BreastCancer'
test_generator = mask_bounding_box_predictor_generator(mode, dataset, shuffle=True, normalise=False, 
                                                       shift_centres=True, shift_sizes=False, batch_size=batch_size, 
                                                       use_radiomics=True)
num_bg = 0.0
num_mask = 0.0

for i in range(1):
    ctpet, mask = next(test_generator)
       
    print(ctpet.shape)
        
    unique, counts = np.unique(mask, return_counts=True)
    counts = dict(zip(unique, counts))
    
    num_bg += counts[0]
    num_mask += counts.get(1,0)
                      
print('mask proportion:', num_mask / num_bg)

(10, 32, 32, 32, 16)
mask proportion: 0.0015282107708295127


In [23]:
# calculates output shape given input parameters

def deconv_calculator(input_size, kernels, stride):
    input_size = np.array(input_size)
    stride = np.array(stride)

    for kernel in kernels:
        
        kernel = np.array(kernel)
        input_size = np.ceil((input_size - 1) * stride + kernel)
        print(input_size)

deconv_calculator([3,3,3], [[3,3,5],
                            [3,3,5],
                            [5,5,5],
                            [5,5,5],
                           [4,5,5]], [1,1,1])


[5. 5. 7.]
[ 7.  7. 11.]
[11. 11. 15.]
[15. 15. 19.]
[18. 19. 23.]


In [24]:
from sklearn.utils import class_weight

#computes the weights where if they are applied to each item, taking the mean would yield 1
#used for segmentation mask weighting
def get_class_weight(mask_proportion=0.33, precision=1000):
    
    arr = np.empty(precision)
    index = int(mask_proportion * precision)
    arr[:index] = 1
    arr[index:] = 0
    weights = class_weight.compute_class_weight('balanced', [0,1], arr)
    return weights

class_weights = get_class_weight(0.25996801724912405) #proportion taken from above
print(class_weights)

[0.67476383 1.93050193]


In [25]:
from scipy.spatial import distance
import keras.backend as K
import matplotlib.pyplot as plt

#Accepts tensors; from https://github.com/keras-team/keras/issues/3611
#Higher is better
def dice_metric(y_true, y_pred, smooth=1):
    y_pred = K.round(y_pred)

    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

#From https://gist.github.com/wassname/7793e2058c5c9dacb5212c0ac0b18a8a
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    y_true = K.batch_flatten(y_true)
    y_pred = K.batch_flatten(y_pred)
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return K.mean((2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


#From https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    y_true = K.batch_flatten(y_true)
    y_pred = K.batch_flatten(y_pred)
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = K.mean((intersection + smooth) / (sum_ - intersection + smooth))
    return (1 - jac) * smooth



#from https://stackoverflow.com/questions/46009619/keras-weighted-binary-crossentropy
def weighted_binary_crossentropy(y_true, y_pred):
    zero_weight = class_weights[0]
    one_weight = class_weights[1]
    
    # Calculate the binary crossentropy
    b_ce = K.binary_crossentropy(y_true, y_pred)
    
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce

    # Return the mean error
    return K.mean(weighted_b_ce)

def blyat():

    # Test
    # Test
    print("TYPE                 |Almost_right |half right |all_wrong")
    y_true = np.array([[1,0,0,0,0,0,0,0,0,0,0]])
    y_pred = np.array([[1,0,0,0,0,0,0,0,0,0,0]])
    
    r = (
        K.variable(y_true) *
        K.variable(y_pred)
    ).eval(session=K.get_session())
    print('mult',r)
    
    r = jaccard_distance_loss(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('jaccard_distance_loss',r)
    #assert r[0]<r[1]
    #assert r[1]<r[2]

    r = keras.losses.binary_crossentropy(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('binary_crossentropy',r)
    print('binary_crossentropy_scaled',r/r.max())
    #assert r[0]<r[1]
    #assert r[1]<r[2]
    
    r = weighted_binary_crossentropy(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('weighted_binary_crossentropy',r)
    print('weighted_binary_crossentropy_scaled',r/r.max())
    
    ups = np.arange(8).reshape((1,2,2,2))
    r = resnet3d3.upsample3d(2)(
        K.variable(ups)
    ).eval(session=K.get_session())
    print('upsampled',r)
    
blyat()

TYPE                 |Almost_right |half right |all_wrong
mult [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
jaccard_distance_loss 0.0
binary_crossentropy [1.0174631e-07]
binary_crossentropy_scaled [1.]
weighted_binary_crossentropy 8.226345e-08
weighted_binary_crossentropy_scaled 1.0
upsampled [[[[0. 0. 1. 1.]
   [0. 0. 1. 1.]
   [2. 2. 3. 3.]
   [2. 2. 3. 3.]]

  [[0. 0. 1. 1.]
   [0. 0. 1. 1.]
   [2. 2. 3. 3.]
   [2. 2. 3. 3.]]

  [[4. 4. 5. 5.]
   [4. 4. 5. 5.]
   [6. 6. 7. 7.]
   [6. 6. 7. 7.]]

  [[4. 4. 5. 5.]
   [4. 4. 5. 5.]
   [6. 6. 7. 7.]
   [6. 6. 7. 7.]]]]


In [37]:
import utils
from scipy.ndimage.morphology import binary_fill_holes

#Evaluate segmentation model

#validation_generator = mask_bounding_box_predictor_generator('test', dataset, shuffle=False, shift_centres=False, shift_sizes=False, batch_size=batch_size)
#print(segmentation_model.evaluate_generator(validation_generator, steps=math.ceil(len(ct_data['train'][dataset])/batch_size)))

#Accepts numpy arrays
#Higher is better
def dice_eval(true_mask, pred_mask):
    pred_mask = np.rint(pred_mask)
    true_mask = np.rint(true_mask)
    
    true_values = np.unique(true_mask)
    pred_values = np.unique(pred_mask)
    
    
    if np.array_equal(true_values, [0]) and np.array_equal(pred_values, [0]):
        return 1
    
    pred_mask = pred_mask.flatten()
    true_mask = true_mask.flatten()
    ret = 1 - distance.dice(pred_mask, true_mask)
    return ret

#returns dice acc, slice-by-slice acc and best image segmentation
def evaluate_segmentation_model(dataset, model, mode='test', output_segmentation=False, fill_holes=False):
    
    validation_generator = mask_bounding_box_predictor_generator(
                    mode, dataset, shuffle=False, shift_centres=False, shift_sizes=False, use_radiomics=False, normalise=True, batch_size=1)
    
    dice_sum = 0
    max_acc = 0
    max_patient = ''
    
    dice_slice_sum = 0
    
    for i in range(len(ct_data[mode][dataset])):
        ctpet, true_mask = next(validation_generator)
        true_mask = true_mask[0]
        pred_mask = model.predict(ctpet)[0]
        
        if fill_holes:
            pred_mask = binary_fill_holes(pred_mask)
        
        dice_index = dice_eval(true_mask, pred_mask)
        dice_sum += dice_index
                
        if dice_index > max_acc:
            max_acc = dice_index
            max_patient = ct_data[mode][dataset][i]
        
        if output_segmentation:
            patient_name = os.path.splitext(os.path.basename(pet_data[mode][dataset][i]))[0]
            mask_radius = 0.5 * np.array((32, 32, 32))
            utils.output_numpy_mask_to_nrrd(patient_name, np.rint(pred_mask), centre_data[mode][dataset][i], mask_radius, dataset)
            utils.output_numpy_mask_to_nrrd(patient_name, true_mask, centre_data[mode][dataset][i], mask_radius, dataset, filename_tag='-true')
            
        temp_slice_sum = 0
        for j in range(pred_mask.shape[2]):
            temp_slice_sum += dice_eval(true_mask[:,:,j], pred_mask[:,:,j])
        
        dice_slice_sum += temp_slice_sum / pred_mask.shape[2]
        
        
    return dice_sum / len(ct_data[mode][dataset]), dice_slice_sum / len(ct_data[mode][dataset]), os.path.basename(max_patient), max_acc

#print('dice accuracy, dice slice-by-slice accuracy, most accurate:')
print(evaluate_segmentation_model('BreastCancer', segmentation_model, 'test', output_segmentation=False
                                 ))




(0.735979651667195, 0.7131510984944008, '10790677.npy', 0.9461363390050938)


In [34]:
reset_keras()

1713576


In [35]:
from keras.models import load_model


#cycles_per_epoch = how many times the entire training set should be cycled over for each epoch
#total_cycles = how many times the entire training set should be cycled in total
def train_segmentation_model(dataset, batch_size=4, total_epochs=50, shift_centres=False, shift_sizes=False, shuffle=True, 
                             normalise=True, save_model=True, use_radiomics=True, transferring_model=None, loss=jaccard_distance_loss,
                            es_patience=200):
    
    def get_model_name():
        return 'segmentation {} batch={} cycles={} shift_centre={} shift_size={} shuffle={} normalise={} use_radiomics={} transferred={} {}.h5'.format(
            dataset, batch_size, total_epochs, shift_centres, shift_sizes, shuffle, normalise, use_radiomics, transferring_model!=None, loss.__name__)

    #bounding box values derived from NumpyAnalysis.ipynb
    if use_radiomics:
        segmentation_input_shape = (32, 32, 32, 16)
    else:
        segmentation_input_shape = (32, 32, 32, 2)
    segmentation_output_shape = (32, 32, 32)
    
    train_generator = mask_bounding_box_predictor_generator(
        'train', dataset, shuffle=shuffle, shift_centres=shift_centres, shift_sizes=shift_sizes, 
        normalise=normalise, use_radiomics=use_radiomics, batch_size=batch_size)
    validation_generator = mask_bounding_box_predictor_generator(
        'test', dataset, shuffle=False, shift_centres=False, shift_sizes=False, 
        normalise=normalise, use_radiomics=use_radiomics, batch_size=batch_size)


    #copy all layer data over until the avg-pool/dense layer in the detection model; indices are found with model.summary()
    #this is terrible :|
    #for detect_layer, segment_layer in zip(detection_model.layers[1:-4], segmentation_model.layers[1:]):
    #    segment_layer.set_weights(detect_layer.get_weights())

    if transferring_model is not None:
        segmentation_model = transferring_model
        #for source_layer, target_layer in zip(transferring_model.layers, segmentation_model.layers):
        #    target_layer.set_weights(source_layer.get_weights())
    else:
        segmentation_model = Resnet3DBuilder.build_resnet_18(segmentation_input_shape, segmentation_output_shape, mode='segmentation')
        segmentation_model.compile(optimizer='adam',
                      loss=[loss],
                        metrics=[dice_coef, dice_metric])
        
    es = EarlyStopping(monitor='val_loss', patience=es_patience, restore_best_weights=True)
    history = segmentation_model.fit_generator(train_generator, validation_data=validation_generator, verbose=1,
                                             validation_steps=math.ceil(len(ct_data['test'][dataset])/batch_size), 
                                             epochs=total_epochs, steps_per_epoch=math.ceil(len(ct_data['train'][dataset])/batch_size), 
                                            callbacks=[es])
    
    loss_hist = history.history['loss']
    val_hist = history.history['val_loss']
    dice_index,_,_,_ = evaluate_segmentation_model(dataset, segmentation_model)
    
    with np.printoptions(precision=2):
        print('loss:', np.array(loss_hist))
        print('val_loss:', np.array(val_hist))
        print('dice_index:', dice_index)
        
    with open('Models/keras models/training_log.txt', 'a+') as log:
        log.write('{} \ndice index: {} \ntraining: {}\nvalidation: {}\n\n'.format(get_model_name(), dice_index, loss_hist, val_hist))
    
    if save_model:
        model_name = get_model_name()
        segmentation_model.save('Models/keras models/' + model_name)
        
        
    return segmentation_model

'''segmentation_model = train_segmentation_model('HeadNeckCancer', batch_size=4, total_epochs=100,
                                              shift_centres=False, shift_sizes=False, normalise=True, shuffle=True, use_radiomics=False)'''

segmentation_model = train_segmentation_model('BreastCancer', batch_size=4, total_epochs=100,
                                              shift_centres=False, shift_sizes=False, normalise=True, shuffle=True, use_radiomics=False)

'''segmentation_model = train_segmentation_model('BreastCancer', batch_size=4, total_epochs=100,
                                              shift_centres=False, shift_sizes=False, normalise=True, shuffle=True, use_radiomics=False,
    transferring_model=load_model('/home/jzhe0882/Models/keras models/segmentation HeadNeckCancer batch=4 cycles=100 shift_centre=False shift_size=False shuffle=True normalise=True use_radiomics=False transferred=False jaccard_distance_loss.h5', 
                                custom_objects={'jaccard_distance_loss' : jaccard_distance_loss,
                                    'dice_coef_loss': dice_coef_loss,
                                          'dice_metric': dice_metric,
                                          'dice_coef': dice_coef,
                                       'conv4d' : conv4d.conv4d,
                                    'l2': l2,
                                    'Resnet3DBuilder': Resnet3DBuilder,
                                    'tf' : tf,
                                    'upsample3d_helper' : resnet3d3.upsample3d_helper},
                                 compile=True))'''

Model number for session: 8
[<tf.Tensor 'multiply_1/mul:0' shape=(?, 4, 4, 4, 1024) dtype=float32>, <tf.Tensor 'multiply_2/mul:0' shape=(?, 8, 8, 8, 512) dtype=float32>, <tf.Tensor 'multiply_3/mul:0' shape=(?, 16, 16, 16, 256) dtype=float32>, <tf.Tensor 'multiply_4/mul:0' shape=(?, 32, 32, 32, 128) dtype=float32>]
Tensor("conv3d_42/add:0", shape=(?, 8, 8, 8, 256), dtype=float32)
Tensor("conv3d_44/add:0", shape=(?, 8, 8, 8, 256), dtype=float32)
Tensor("conv3d_46/add:0", shape=(?, 16, 16, 16, 128), dtype=float32)
Tensor("conv3d_48/add:0", shape=(?, 16, 16, 16, 128), dtype=float32)
Tensor("conv3d_50/add:0", shape=(?, 32, 32, 32, 64), dtype=float32)
Tensor("conv3d_52/add:0", shape=(?, 32, 32, 32, 64), dtype=float32)
Tensor("activation_51/Relu:0", shape=(?, 32, 32, 32, 192), dtype=float32)
Epoch 1/100
27/27 [==============================] - 41s 2s/step - loss: 57.6628 - dice_coef: 0.6886 - dice_metric: 0.6558 - val_loss: 54.1623 - val_dice_coef: 0.6982 - val_dice_metric: 0.6714
Epoch 2/100

"segmentation_model = train_segmentation_model('BreastCancer', batch_size=4, total_epochs=100,\n                                              shift_centres=False, shift_sizes=False, normalise=True, shuffle=True, use_radiomics=False,\n    transferring_model=load_model('/home/jzhe0882/Models/keras models/segmentation HeadNeckCancer batch=4 cycles=100 shift_centre=False shift_size=False shuffle=True normalise=True use_radiomics=False transferred=False jaccard_distance_loss.h5', \n                                custom_objects={'jaccard_distance_loss' : jaccard_distance_loss,\n                                    'dice_coef_loss': dice_coef_loss,\n                                          'dice_metric': dice_metric,\n                                          'dice_coef': dice_coef,\n                                       'conv4d' : conv4d.conv4d,\n                                    'l2': l2,\n                                    'Resnet3DBuilder': Resnet3DBuilder,\n                         

In [13]:
segmentation_model = load_model('/home/jzhe0882/Models/keras models/segmentation BreastCancer batch=8 cycles=100 shift_centre=False shift_size=False shuffle=True normalise=True use_radiomics=False transferred=True jaccard_distance_loss.h5',
                               custom_objects={'jaccard_distance_loss' : jaccard_distance_loss,
                                    'dice_coef_loss': dice_coef_loss,
                                          'dice_metric': dice_metric,
                                          'dice_coef': dice_coef})

Instructions for updating:
Use tf.cast instead.


In [12]:
from keras.models import load_model

segmentation_model=load_model('Models/keras models/segmentation BreastCancer False 50 8 True True.h5',
                           custom_objects={'dice_coef_loss': dice_coef_loss,
                                          'dice_metric': dice_metric,
                                          'dice_coef': dice_coef})



Instructions for updating:
Use tf.cast instead.


F

# Playground

In [ ]:
def blyat(arr):
    return np.roll(arr, [2,1], [0,1])
    
test = np.arange(20).reshape(4,5)
print(test)
test=blyat(test)
print(test)
test[-1:, :] = 0
print(test)

In [ ]:
test = np.arange(1,31).reshape(2,3,5)
print(test)
centre = [1,1,1]
print(test[centre[0], centre[1], centre[2]])
test = get_bounding_box(test, np.array(centre), np.array([2,2,2]))
print(test)
test = align_bounding_box(test, np.array(centre), np.array([3,3,3]))
print(test)


In [ ]:
print(detection_model.get_layer(index=-4))

In [58]:
#resample bounding_box to target_size while keeping aspect ratios
def resize_bounding_box2(bounding_box, target_size, mode):
    size_ratio = np.amin(np.divide(target_size, bounding_box.shape))
    size_ratio_index = np.argmin(np.divide(target_size, bounding_box.shape))
    
    #the resampling here is slightly inaccurate since new_size is rounded to the nearest int
    bounding_box = sitk.GetImageFromArray(np.transpose(bounding_box))
    new_size = np.rint(size_ratio * np.array(bounding_box.GetSize()))
    new_spacing = tuple(np.divide(bounding_box.GetSpacing(), size_ratio))
            
    assert new_size[size_ratio_index] == target_size[size_ratio_index]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(bounding_box)
    resampler.SetSize((int(new_size[0]), int(new_size[1]), int(new_size[2])))
    resampler.SetOutputSpacing(new_spacing)

    if mode == 'mask':
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
        
    new_box = np.transpose(sitk.GetArrayFromImage(resampler.Execute(bounding_box)))
    
    #pads the surroundings with 0's
    return get_bounding_box(new_box, 0.5 * new_size, 0.5 * target_size)

#resample bounding_box to target_size while keeping aspect ratios
#alternative to resize_bounding_box that doesnt centre the volume
def resize_bounding_box3(bounding_box, target_size, mode):
    size_ratio = np.amin(np.divide(target_size, bounding_box.shape))
    size_ratio_index = np.argmin(np.divide(target_size, bounding_box.shape))
    
    #the resampling here is slightly inaccurate since new_size is rounded to the nearest int
    bounding_box = sitk.GetImageFromArray(np.transpose(bounding_box))
    new_size = target_size
    new_spacing = tuple(np.divide(bounding_box.GetSpacing(), size_ratio))
        
    assert new_size[size_ratio_index] == target_size[size_ratio_index]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(bounding_box)
    resampler.SetSize((int(new_size[0]), int(new_size[1]), int(new_size[2])))
    resampler.SetOutputSpacing(new_spacing)

    if mode == 'mask':
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
        
    new_box = resampler.Execute(bounding_box)
    new_box = np.transpose(sitk.GetArrayFromImage(resampler.Execute(new_box)))
        
    
    return new_box


test = np.array([[[2,1,2,4],
                  [2,1,2,4],
                  [2,1,2,4]], 
                 
                 [[2,1,2,4],
                  [2,1,3,4],
                  [2,1,2,4]]], dtype=float)

#test = np.arange(1,1001).reshape((10,10,10)).astype(float)
print(test.shape)
test = resize_bounding_box3(test, np.array([4,6,8]), 'mask')
#test = resize_bounding_box3(test, np.array([2,3,4]), 'mask')
print(test.shape)
print(test)


(2, 3, 4)
(4, 6, 8)
[[[2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 3. 3. 4. 4. 0.]
  [2. 1. 1. 3. 3. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 3. 3. 4. 4. 0.]
  [2. 1. 1. 3. 3. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [2. 1. 1. 2. 2. 4. 4. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


In [31]:
#old validation generator for resizing stuff
maximal_bounding_volume = np.array([32,32,32])

#generator for (ct bounding box, pet bounding_box)-> mask bounding box prediction models
def mask_bounding_box_predictor_generator(mode='train', dataset='HeadNeckCancer', shift_centres=False, shift_sizes=False, shift_weight=0.7, 
                                          shuffle=False, normalise=True, use_radiomics=True, batch_size=4, region_radius=[16,16,16]):

    def shift_centre(centre):
        centre = np.random.randn(3) * shift_weight * np.array([20.4,4.6,21.2]) + centre #std taken from NumpyAnalysis
        centre = np.maximum(centre, [0,0,0]) #keep centre within image bounds
        centre = np.minimum(centre, volume_size)
        return centre.astype(int)
    
    def shift_size(size):
        size = np.random.randn(3) * shift_weight *  np.array([3.6,3,5.7]) + size #std taken from NumpyAnalysis
        size = np.maximum(size, [1,1,1]) #keep size to > 1
        size = np.minimum(size, maximal_bounding_volume)
        return size
        
    def get_resized_bounding_box(source_volume, centre, size, target_size, mode):
        bounding_box = get_bounding_box(source_volume, centre, 0.5 * size)
        return resize_bounding_box(bounding_box, target_size, mode)
            
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, mask_files, centres, sizes, 
               shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size):
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files = zip(*z)
                                
            ct_pet_batch = []
            mask_batch = []
            
            for i in range(len(pet_files)):
                
                #randomly sample centres/sizes around the true centre/sizes to generate bounding boxes
                if shift_centres:
                    centre = shift_centre(centres[i])
                else:
                    centre = centres[i]

                if shift_sizes:
                    size = shift_size(sizes[i])
                else:
                    size = sizes[i]
  

                ct = get_resized_bounding_box(np.load(ct_files[i]), centre, size, maximal_bounding_volume, 'ct')
                pet = get_resized_bounding_box(np.load(pet_files[i]), centre, size, maximal_bounding_volume, 'pet')
                mask = get_resized_bounding_box(np.load(mask_files[i]), centre, size, maximal_bounding_volume, 'mask')
                
                ct_pet = zip_np_volumes(ct, pet)
                
                if use_radiomics:
                    ct_radiomics = np.load(ct_radiomics_files[i])
                    pet_radiomics = np.load(pet_radiomics_files[i])
                    
                    for axis in range(ct_radiomics.shape[-1]):
                        ct_radiomic_feature = get_resized_bounding_box(ct_radiomics[:,:,:,axis], centre, size, maximal_bounding_volume, 'ct')
                        pet_radiomic_feature = get_resized_bounding_box(pet_radiomics[:,:,:,axis], centre, size, maximal_bounding_volume, 'pet')
                        ct_radiomic_feature = np.expand_dims(ct_radiomic_feature, axis=-1)
                        pet_radiomic_feature = np.expand_dims(pet_radiomic_feature, axis=-1)
                        
                        ct_pet = np.concatenate((ct_pet, ct_radiomic_feature, pet_radiomic_feature), axis=-1)
                
                if normalise:
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                mask_batch.append(mask)

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(mask_batch)
                    ct_pet_batch.clear()
                    mask_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(mask_batch)
                
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  mask_data[mode][dataset], centre_data[mode][dataset], size_data[mode][dataset],
                 shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size)